### This notebook's purpose is to get the CALCE data in a workable format (in a dataframe, cleaned, battery type labeled etc.)

### Goals: (Add to this)
1. Get files from CALCE website into dataframe(s)
2. Add label for every cycle which battery this data came from 
3. Find/Remove empty cells  (delete the whole row or should we do interpolation?) (Might not even have empty cells)


All CS2 cells underwent the same charging profile which was a standard constant current/constant voltage protocol with a constant current rate of 0.5C until the voltage reached 4.2V and then 4.2V was sustained until the charging current dropped to below 0.05A. Unless specified, the discharge cut off voltage for these batteries was 2.7V. All the CS2 cells were randomly numbered and named accordingly. Name 'CS2_n' was given for the nth numbered CS2 cell. 

#### Imports:

In [4]:
import scipy.io
import scipy.signal
import os 
import pandas as pd
from pandas import ExcelWriter
#also had to pip install openpyxl
import zipfile
import requests
import xlrd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import statistics as stats
from math import isclose
from pathlib import Path
%matplotlib inline

#### Notes:
To import .mat files into python:  
import scipy.io   
mat = scipy.io.loadmat('file.mat')  

To unzip files: look at SEDS HW2 (not sure if we will use this)


#### CS2 data (in excel format*)  
Look at CS2 data first. From the website, CS2_'n' was given to the nth numbered CS2 cell (identifies unique cells from eachother). Each cell was cycled multiple times. Excel files are named using the testing dates.  **Were the same cells cycled on different dates or was the continuous cycling data split into multiple excel files?**   

*CS2_8 and CS2_21 are in .txt format because they were cycled using a different system.

Starting with CS2_33, in excel file format

### From Robert's Notebook + Sarah's Edits:

In [5]:
import glob 
import os 
import pandas as pd
import requests

def get_data(filepath): 
    """Imports all data in given path"""
    rootdir = filepath
    file_list = [f for f in glob.glob(os.path.join(rootdir,'*.xlsx'))] #iterate through dir to get excel files 
    
    d = {} #initiate dict for data storage
    count = 0
    for file in file_list:
        count += 1
        name = os.path.split(file)[1].split('.')[0]
        data = pd.read_excel(file,1)
        new_set = {name : data}
        d.update(new_set)
        print("adding file " + str(count) + ' ' + str(name))
    return d
### There are 23 files in the CS2 directory, so we should have 23 entries in the dictionary - add unit test for this, super EASY check 


In [6]:
d = get_data('data/CS2_33/')

adding file 1 CS2_33_10_04_10
adding file 2 CS2_33_10_05_10
adding file 3 CS2_33_10_15_10
adding file 4 CS2_33_10_26_10
adding file 5 CS2_33_11_01_10
adding file 6 CS2_33_11_10_10
adding file 7 CS2_33_11_19_10
adding file 8 CS2_33_11_24_10
adding file 9 CS2_33_12_08_10
adding file 10 CS2_33_12_16_10
adding file 11 CS2_33_12_23_10
adding file 12 CS2_33_1_10_11
adding file 13 CS2_33_1_18_11
adding file 14 CS2_33_1_24_11
adding file 15 CS2_33_1_28_11
adding file 16 CS2_33_2_2_11
adding file 17 CS2_33_8_17_10
adding file 18 CS2_33_8_18_10
adding file 19 CS2_33_8_19_10
adding file 20 CS2_33_8_30_10
adding file 21 CS2_33_9_17_10
adding file 22 CS2_33_9_27_10
adding file 23 CS2_33_9_7_10


In [9]:
data1 = d['CS2_33_10_04_10']

In [10]:
data1['Cycle_Index'].max()

23

In [49]:
#iterate over keys of data to add a battery label and a column of None dV values
def add_label_dv_cols(dictionary):
    '''This adds the battery label based off of the keys in the input dictionary, and adds an empty dV column and an empty dQ/dV column'''     
    for keys in dictionary:
        data1 = dictionary[keys]
        # add a battery label corresponding to that battery's key. 
        data1['Battery_Label'] = keys
        #create column labeled 'dV'
        data1['dV'] = None
        data1['dQ/dV'] = None 
    return

In [12]:
add_label_dv_cols(dictionary = d)

In [13]:
#separate out dataframes into cycles
def sep_cycles(dataframe):
    """This function separates out the cycles in the battery dataframe by grouping by the 'Cycle_Index' column, and putting them in a dictionary. """
    gb = dataframe.groupby(by = ['Cycle_Index'])
    cycle_dict = dict(iter(gb))
    return cycle_dict

In [14]:
test = sep_cycles(data1)
#test is a dictionary of cycles

In [15]:

#writer = ExcelWriter('NLToutput.xlsx')
#test[1].to_excel(writer)
#writer.save()

In [16]:
def save_sep_cycles_xlsx(cycle_dict, battname, path_to_folder):
    '''This saves the separated out cycles into different excel files, beginning with the battery name. Battname and path to folder must be strings.'''
    for i in range(1,len(cycle_dict)+1):
        writer = ExcelWriter(path_to_folder + battname + 'Cycle' + str(i) + '.xlsx')
        cycle_dict[i].to_excel(writer)
        writer.save()
    return 


In [20]:
#save_sep_cycles_xlsx(test, 'NLTtest', 'NLT_Test/')

### Calculate dV and dq/dv and Clean Data 

In [17]:
def calc_dv_dqdv(cycle_df):
    '''This function calculates the dv and the dq/dv for a dataframe.'''
    cycle_df = cycle_df.reset_index(drop = True)
    for i in range(1,len(cycle_df)): 
        cycle_df.loc[i, ('dV')] = cycle_df.loc[i, ('Voltage(V)')] - cycle_df.loc[i-1, ('Voltage(V)')]
    #calculate dq/dv based off of discharge capacity - might change this later so user can choose to use charge or discharge cap. 
    cycle_df['dQ/dV'] = cycle_df['Discharge_Capacity(Ah)']/cycle_df['dV']
    return cycle_df

In [18]:
data2 = calc_dv_dqdv(data1)

In [19]:
data2.head()

,Data_Point,Test_Time(s),Date_Time,Step_Time(s),Step_Index,Cycle_Index,Current(A),Voltage(V),Charge_Capacity(Ah),Discharge_Capacity(Ah),Charge_Energy(Wh),Discharge_Energy(Wh),dV/dt(V/s),Internal_Resistance(Ohm),Is_FC_Data,AC_Impedance(Ohm),ACI_Phase_Angle(Deg),Battery_Label,dV,dQ/dV
0,1,30.000116,2010-09-27 14:12:48,30.000117,1,1,0.000000,3.451826,0.000000,0.0,0.000000,0.0,-0.000032,0.0,0,0,0,CS2_33_10_04_10,None,NaN
1,2,60.015409,2010-09-27 14:13:18,60.015410,1,1,0.000000,3.451826,0.000000,0.0,0.000000,0.0,0.000000,0.0,0,0,0,CS2_33_10_04_10,0,NaN
2,3,90.030676,2010-09-27 14:13:48,90.030677,1,1,0.000000,3.451988,0.000000,0.0,0.000000,0.0,0.000000,0.0,0,0,0,CS2_33_10_04_10,0.000161886,0
3,4,120.014848,2010-09-27 14:14:18,120.014848,1,1,0.000000,3.451826,0.000000,0.0,0.000000,0.0,-0.000032,0.0,0,0,0,CS2_33_10_04_10,-0.000161886,-0
4,5,150.030659,2010-09-27 14:14:48,30.015332,2,1,0.550025,3.584821,0.004585,0.0,0.016314,0.0,0.001264,0.0,0,0,0,CS2_33_10_04_10,0.132995,0


In [20]:
def drop_0_dv(cycle_df_dv): 
    '''Drop rows where dv=0 (or about 0) in a dataframe that has already had dv calculated. Then recalculate dv and dq/dv'''
    #this will clean up the data points around V = 4.2V (since they are holding it at 4.2V for a while).
    cycle_df_dv = cycle_df_dv.dropna(subset=['dQ/dV'])
    cycle_df_dv = cycle_df_dv.reset_index(drop = True)
    for i in range(1, len(cycle_df_dv)):
        if isclose(cycle_df_dv.loc[i, ('dV')], 0, abs_tol = 10**-3):
            cycle_df_dv = cycle_df_dv.drop(index = i)
    #reset index
    cycle_df_dv = cycle_df_dv.reset_index(drop = True)
    #recalculating dv after dropping rows
    for i in range(1, len(cycle_df_dv)): 
        cycle_df_dv.loc[i, ('dV')] = cycle_df_dv.loc[i, ('Voltage(V)')] - cycle_df_dv.loc[i-1, ('Voltage(V)')]
    #recalculate dq/dv  
    cycle_df_dv['dQ/dV'] = cycle_df_dv['Discharge_Capacity(Ah)']/cycle_df_dv['dV']
    return cycle_df_dv  

In [21]:
data3 = drop_0_dv(cycle_df_dv=data2)

In [22]:
data3.tail()

,Data_Point,Test_Time(s),Date_Time,Step_Time(s),Step_Index,Cycle_Index,Current(A),Voltage(V),Charge_Capacity(Ah),Discharge_Capacity(Ah),Charge_Energy(Wh),Discharge_Energy(Wh),dV/dt(V/s),Internal_Resistance(Ohm),Is_FC_Data,AC_Impedance(Ohm),ACI_Phase_Angle(Deg),Battery_Label,dV,dQ/dV
9753,10755,356498.076170,2010-10-01 17:14:23,750.381752,7,23,-0.549993,3.983482,24.624331,23.690529,97.876133,88.765393,-0.000065,0.09335,0,0,0,CS2_33_10_04_10,-0.00372601,-6358.16
9754,10756,356528.091502,2010-10-01 17:14:53,780.397083,7,23,-0.550173,3.979756,24.624331,23.695116,97.876133,88.783657,-0.000065,0.09335,0,0,0,CS2_33_10_04_10,-0.00372577,-6359.8
9755,10757,356558.106839,2010-10-01 17:15:23,810.412421,7,23,-0.550173,3.976031,24.624331,23.699703,97.876133,88.801903,-0.000097,0.09335,0,0,0,CS2_33_10_04_10,-0.00372577,-6361.03
9756,10758,356588.107248,2010-10-01 17:15:53,840.412830,7,23,-0.550173,3.972305,24.624331,23.704288,97.876133,88.820123,-0.000097,0.09335,0,0,0,CS2_33_10_04_10,-0.00372577,-6362.26
9757,10759,356618.121556,2010-10-01 17:16:23,870.427138,7,23,-0.550173,3.968579,24.624331,23.708875,97.876133,88.838334,-0.000129,0.09335,0,0,0,CS2_33_10_04_10,-0.00372577,-6363.49


### Separate by charge/discharge 
#### We want to take each cycle as an individual, but we also want to separate into charge+discharge just to do gaussian fitting easier, but still take charge+discharge as one set. So after cycles are separated out to be treated as individuals, we will run above code to smooth, separate them into charge and discharge (could reverse those two steps) and then do gaussian fitting to find peak descriptors. 

In [23]:
data4 = pd.read_excel('NLT_Test/NLTtestCycle11.xlsx')
#as an example of one cycle

In [24]:
data5 = calc_dv_dqdv(data4)

In [33]:
data5

,Data_Point,Test_Time(s),Date_Time,Step_Time(s),Step_Index,Cycle_Index,Current(A),Voltage(V),Charge_Capacity(Ah),Discharge_Capacity(Ah),Charge_Energy(Wh),Discharge_Energy(Wh),dV/dt(V/s),Internal_Resistance(Ohm),Is_FC_Data,AC_Impedance(Ohm),ACI_Phase_Angle(Deg),Battery_Label,dV,dQ/dV
0,4771,156587.110342,2010-09-29 09:42:17,30.015269,1,11,0.000000,3.156191,10.689637,10.700286,42.453113,40.078347,0.000583,0.095613,0,0,0,CS2_33_10_04_10,NaN,NaN
1,4772,156617.125821,2010-09-29 09:42:47,60.030749,1,11,0.000000,3.173686,10.689637,10.700286,42.453113,40.078347,0.000421,0.095613,0,0,0,CS2_33_10_04_10,0.017495,611.614251
2,4773,156647.140885,2010-09-29 09:43:17,90.045813,1,11,0.000000,3.186970,10.689637,10.700286,42.453113,40.078347,0.000356,0.095613,0,0,0,CS2_33_10_04_10,0.013283,805.547147
3,4774,156677.109273,2010-09-29 09:43:47,120.014200,1,11,0.000000,3.197175,10.689637,10.700286,42.453113,40.078347,0.000292,0.095613,0,0,0,CS2_33_10_04_10,0.010206,1048.481573
4,4775,156707.129010,2010-09-29 09:44:17,30.015288,2,11,0.549845,3.421047,10.694221,10.700286,42.468538,40.078347,0.002527,0.095613,0,0,0,CS2_33_10_04_10,0.223872,47.796357
5,4776,156737.144258,2010-09-29 09:44:47,60.030537,2,11,0.549845,3.496536,10.698806,10.700286,42.484407,40.078347,0.001717,0.095613,0,0,0,CS2_33_10_04_10,0.075488,141.748006
6,4777,156767.159538,2010-09-29 09:45:17,90.045816,2,11,0.549845,3.551289,10.703390,10.700286,42.500568,40.078347,0.001296,0.095613,0,0,0,CS2_33_10_04_10,0.054753,195.428079
7,4778,156797.174807,2010-09-29 09:45:47,120.061086,2,11,0.549845,3.594378,10.707975,10.700286,42.516952,40.078347,0.001004,0.095613,0,0,0,CS2_33_10_04_10,0.043090,248.324888
8,4779,156827.190139,2010-09-29 09:46:17,150.076417,2,11,0.549845,3.630178,10.712560,10.700286,42.533515,40.078347,0.000907,0.095613,0,0,0,CS2_33_10_04_10,0.035800,298.890845
9,4780,156857.205337,2010-09-29 09:46:47,180.091615,2,11,0.549845,3.660633,10.717144,10.700286,42.550230,40.078347,0.000778,0.095613,0,0,0,CS2_33_10_04_10,0.030454,351.354396


In [45]:
def sep_char_dis(df_dqdv):
    '''Takes a dataframe of one cycle with calculated dq/dv and separates into charge and discharge differential capacity curves'''
    charge = df_dqdv[df_dqdv['dV'] < 0] 
    charge = charge.reset_index(drop = True)
    discharge = df_dqdv[df_dqdv['dV'] >= 0] 
    discharge = discharge.reset_index(drop = True)
    return charge, discharge
    

In [46]:
charge, discharge = sep_char_dis(data5)

In [48]:
charge.head()

,Data_Point,Test_Time(s),Date_Time,Step_Time(s),Step_Index,Cycle_Index,Current(A),Voltage(V),Charge_Capacity(Ah),Discharge_Capacity(Ah),Charge_Energy(Wh),Discharge_Energy(Wh),dV/dt(V/s),Internal_Resistance(Ohm),Is_FC_Data,AC_Impedance(Ohm),ACI_Phase_Angle(Deg),Battery_Label,dV,dQ/dV
0,4982,162902.242726,2010-09-29 11:27:32,30.015199,3,11,0.000000,4.116315,11.635910,10.700286,46.189533,40.078347,-0.000389,0.095613,0,0,0,CS2_33_10_04_10,-0.083750,-127.764962
1,4983,162932.258004,2010-09-29 11:28:02,60.030477,3,11,0.000000,4.106110,11.635910,10.700286,46.189533,40.078347,-0.000194,0.095613,0,0,0,CS2_33_10_04_10,-0.010206,-1048.457079
2,4984,162962.273325,2010-09-29 11:28:32,90.045798,3,11,0.000000,4.100440,11.635910,10.700286,46.189533,40.078347,-0.000065,0.095613,0,0,0,CS2_33_10_04_10,-0.005670,-1887.310923
3,4985,162992.241812,2010-09-29 11:29:02,120.014285,3,11,0.000000,4.096714,11.635910,10.700286,46.189533,40.078347,-0.000097,0.095613,0,0,0,CS2_33_10_04_10,-0.003726,-2872.152422
4,4989,162999.601966,2010-09-29 11:29:10,7.359516,4,11,0.822465,4.200065,11.637708,10.700286,46.197081,40.078347,0.000000,0.095613,0,0,0,CS2_33_10_04_10,-0.000162,-66195.064518


### Plot code

New Column- To calculate dq/dv do discharge/charge capacity/ dv. calculate dv first. Ask robert whether to use discharge or charge capacity to calculate this. or do both and compare. Maybe ask user which capacity to use between discharge and charge - depends on system (i.e. Maccor) , which anode or cathode they are looking at etc. 

In [ ]:
df1 = pd.DataFrame() 
for i in range(1,len(data1)):
    if data1.loc[i, ('Cycle_Index')] == 7:
        df1 = df1.append(data1.loc[i])
        
fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
myplot = plt.plot(df1['Voltage(V)'], df1['dQ/dV'])

### Filter Sav Golay

In [ ]:
data1['dV'].describe()

In [ ]:
unfilt = pd.concat([data1['dQ/dV-dis']])

In [ ]:
unfiltar = unfilt.values
#converts into an array 

In [ ]:
data1['filtered dq/dv'] = scipy.signal.savgol_filter(unfiltar, 21, 3)

In [ ]:
df1 = pd.DataFrame() 
for i in range(1,len(data1)):
    if data1.loc[i, ('Cycle_Index')] == 6:
        df1 = df1.append(data1.loc[i])
        
fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
myplot = plt.plot(df1['Voltage(V)'], df1['filtered dq/dv'])